In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm
from glob import glob

Loading Ehist Files

In [9]:
indir = "/home/algernon/Outputfiles-arch"
ehists = []
filenames = glob(indir+"/EvoHist*")
with tqdm(total=len(filenames)) as pbar:
    for filename in filenames:
        ehists.append(np.load(filename, allow_pickle=True)['arr_0'])
        pbar.update()

np.savez_compressed("ehists", ehists)
len(ehists)

100%|██████████| 565973/565973 [2:52:01<00:00, 54.83it/s]


565973

In [14]:
ehists = np.load("EHISTS/ehists0.npz", allow_pickle=True)['arr_0']

In [5]:
count = 0
k = 0
for ehist in ehists:
    k+=1
    for i in range(len(ehist)):
        if (ehist[i,5]==13 and ehist[i-1,5]!=13 and ehist[i-1,5]!=12) or (ehist[i,9]==13 and ehist[i-1,9]!=13 and ehist[i-1,9]!=12):
            count += 1
            break
#     for i in range(len(ehist)):
#         if (ehist[i,5]==13 and ehist[i-1,5]!=13 and ehist[i-1,5]==12) or (ehist[i,9]==13 and ehist[i-1,9]!=13 and ehist[i-1,9]==12):
#             count += 1
#         break
count,k

IndexError: index 9 is out of bounds for axis 1 with size 9

In [4]:
len(ehists)

19088

In [6]:
np.shape(ehists)

/home/algernon/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(19088,)